# Este notebook obtendrá los datos del tiempo y los guardará en Mongo

In [1]:
from lxml import html
import requests
import json
import pandas as pd

In [2]:
url = "http://www.aemet.es/es/eltiempo/prediccion/municipios"

In [3]:
page = requests.get(url)
tree = html.fromstring(page.text)

In [4]:
#Cargamos el excel con las coordenadas de las localidades
titulo = "municipios-espana-coordenadas-2011.csv"
df = pd.read_csv(titulo,index_col=0)

print df.ix["Castejón"]

          provincia                    comunidad    latitud  longitud  \
municipio                                                               
Castejón     Cuenca         Castilla - La Mancha  40.382278 -2.519848   
Castejón    Navarra  Navarra, Comunidad Foral de  42.167839 -1.690636   

                                          Enlace Google Maps  
municipio                                                     
Castejón   http://maps.google.es/maps?z=14&ll=40.3822778,...  
Castejón   http://maps.google.es/maps?z=14&ll=42.1678393,...  


In [5]:
provincias = dict()
        

In [6]:
#Saco las provincias de la página
provincias_name = tree.xpath('//select[@id="provincia_selector"]/option/text()')
provincias_id = tree.xpath('//select[@id="provincia_selector"]/option/@value')

#Creo un dataFrame con las provincias y su id para la AEMET
#provincias = pd.DataFrame()
for i in range (0,len(provincias_id)-1):
    provincias[provincias_id[i+1]] = {"nombre":provincias_name[i+1]}


In [7]:
print df.ix["Fonfría"]

          provincia        comunidad    latitud  longitud  \
municipio                                                   
Fonfría      Teruel           Aragón  40.995934 -1.084790   
Fonfría      Zamora  Castilla y León  41.636235 -6.137211   

                                          Enlace Google Maps  
municipio                                                     
Fonfría    http://maps.google.es/maps?z=14&ll=40.9959344,...  
Fonfría    http://maps.google.es/maps?z=14&ll=41.636235,-...  


In [8]:
#Voy a obtener un diccionario con todas las localidades de españa y la provincia a la que pertenecen
localidades = dict()
import datetime

for key in provincias:
    url_municipios = "http://www.aemet.es/es/eltiempo/prediccion/municipios?p="+ key
    page_municipio = requests.get(url_municipios)
    tree_municipio = html.fromstring(page_municipio.text)
    localidades_name = tree_municipio.xpath('//select[@id="localidades_selector"]/option/text()')
    localidades_id = tree_municipio.xpath('//select[@id="localidades_selector"]/option/@value')
    for i in range(0,len(localidades_id)-1):
        nombre_municipio = localidades_name[i+1]
        coordenadas = pd.DataFrame()
        nombre_provincia = provincias[key]['nombre'].encode('utf_8')
        #Hago un try porque hay poblaciones que no se encuentran en el excel, habrá que modificarlas a mano
        try:                                    
            
            nombre_coordenadas = df.ix[nombre_municipio.encode('utf_8')]
            if nombre_coordenadas.provincia == nombre_provincia:
                coordenadas = df.ix[nombre_municipio.encode('utf_8')]
        except:
            #print df.ix[nombre_municipio]
            print "falla: " , nombre_municipio , ": ", nombre_provincia
            coordenadas["longitud"]= ""
            coordenadas["latitud"] = ""
            coordenadas["comunidad"] = ""
            
    #creo el objeto json que voy a asignar al diccionario:

        texto = '{ "_id": "' + localidades_id[i+1][-5:] + '" '
        texto += ', "nombe_localidad" : "'+ localidades_name[i+1] + '" '
        texto += ', "id_localidad" : "' + localidades_id[i+1] + '" '
        texto += ', "id_provincia" : "' +  key  + '" '
        texto += ', "nombre_provincia": "' +   provincias[key]['nombre'] + '" '
        if not coordenadas.empty:
            texto += ', "comunidad" : "' +  coordenadas.comunidad.decode('utf_8') + '" '
       
            texto += ', "longitud": "' +  str(coordenadas.longitud) + '"'
       
            texto += ', "latitud": "' + str(coordenadas.latitud) + '"'
        texto += ', "date" : "' + str(datetime.datetime.utcnow()) + '" '
        texto += '}'
   
        
        if localidades_id[i+1][-5:] in localidades:
            localidades[localidades_id[i+1][-5:]].append(texto)
            
        else: 
            localidades[localidades_id[i+1][-5:]] =texto



falla:  Cieza :  Murcia
falla:  Sancti-Spíritus :  Salamanca
falla:  Beranuy :  Huesca
falla:  Molar, El :  Tarragona
falla:  Fonfría :  Zamora
falla:  Villaescusa :  Zamora
falla:  Rebollar :  Soria
falla:  Moreda de Álava/Moreda Araba :  Araba/Álava
falla:  Campillo, El :  Huelva
falla:  Puig de Santa María, el :  València/Valencia
falla:  Torrent :  València/Valencia
falla:  Campillo, El :  Valladolid
falla:  Villanueva de los Infantes :  Valladolid
falla:  Zarza, La :  Valladolid
falla:  Arroyomolinos :  Madrid
falla:  Molar, El :  Madrid
falla:  Sobrado :  León
falla:  Villanueva de los Infantes :  Ciudad Real
falla:  Guadiana del Caudillo :  Badajoz
falla:  Sancti-Spíritus :  Badajoz
falla:  Zarza, La :  Badajoz
falla:  Arroyomolinos :  Cáceres
falla:  Collado de la Vera :  Cáceres
falla:  Rebollar :  Cáceres
falla:  Cieza :  Cantabria
falla:  Villaescusa :  Cantabria
falla:  Cabanes :  Castelló/Castellón
falla:  Serratella, la :  Castelló/Castellón
falla:  Sada :  A Coruña
falla

In [9]:
print localidades[u'28014']

{ "_id": "28014" , "nombe_localidad" : "Arganda del Rey" , "id_localidad" : "arganda-del-rey-id28014" , "id_provincia" : "28" , "nombre_provincia": "Madrid" , "comunidad" : "Madrid, Comunidad de" , "longitud": "-3.4380798", "latitud": "40.3009506", "date" : "2015-07-15 10:39:52.750952" }


In [10]:
import pymongo
from pymongo import MongoClient

In [16]:
client = MongoClient()
client = MongoClient('localhost', 27017)

In [17]:
db = client.AEMET

In [18]:
collection = db['localidades']

In [24]:
%timeit
def update_localidades_mongo(collection, localidades):
    #Inserto las localidades en Mongo. Esto solo hace falta hacerlo una vez, a partir de ahora empiezo con las previsiones
    for clave in localidades:
        #Convertimos el texto a json para que mongo lo acepte
        d = json.loads(localidades[clave])
        del d['_id']
        post_id = collection.update({u"_id" : clave},{"$set" :d},upsert=True) 
    
update_localidades_mongo(collection, localidades)

